# Train

### Library setups and imports

In [1]:
!pip install -r requirements.txt

In [2]:
import os
import wget
import py7zr
import numpy as np
import tensorflow as tf
from pyunpack import Archive
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv1D, MaxPooling1D, BatchNormalization

### Parameters

In [3]:
USE_DOWNLOADED_SET = True
DOWNLOADED_SET_URL = 'https://hkinsley.com/static/downloads/bci/model_data_v2.7z'
DATA_DIR = 'data/mask-iv-openbci' if USE_DOWNLOADED_SET else 'data/personal'
LABELS = {"left": [1, 0, 0], "none": [0, 1, 0], "right": [0, 0, 1]}
INPUT_SHAPE=(-1, 8, 60)

### Data loading

In [4]:
def generate_dataset(is_validation_data):
    dataset = []
    path = os.path.join(DATA_DIR)
    subdir = 'validation_data/' if is_validation_data else 'data/'

    if not os.path.exists(path):
        wget.download(DOWNLOADED_SET_URL, os.path.join(path, '../mask-iv-openbci.7z'))
        with py7zr.SevenZipFile(os.path.join(DATA_DIR, '../mask-iv-openbci.7z'), mode='r') as z:
            z.extractall(path)
        # TODO: Automatically move 'data' and 'validation_data' one 1 level up on downloaded oh...

    for label in LABELS:
        label_dir = os.path.join(path, subdir, label)
        for FILE in os.listdir(label_dir):
            if USE_DOWNLOADED_SET:
                ffts = np.load(os.path.join(DATA_DIR, subdir, label, FILE))
                for fft_data in ffts:
                    dataset.append([fft_data, label])
            else:
                fft_data = np.load(os.path.join(DATA_DIR, FILE))
                dataset.append([fft_data, label])
    return dataset

In [10]:
training = generate_dataset(True)
validation = generate_dataset(True)

train_x = []
train_y = []
for x, y in training:
    train_x.append(x)
    train_y.append(y)
    
val_x = []
val_y = []
for x, y in validation:
    val_x.append(x)
    val_y.append(y)

train_x = np.array(train_x).reshape(INPUT_SHAPE)
val_x = np.array(val_x).reshape(INPUT_SHAPE)

train_y = np.array(train_y)
val_y = np.array(val_y)

### Neural network definition

In [11]:
model = Sequential([
    Conv1D(32, (3), activation=tf.nn.relu, input_shape=train_x.shape[1:]),
    Conv1D(32, (2), activation=tf.nn.relu),
    MaxPooling1D(pool_size=(2)),
    Conv1D(32, (2), activation=tf.nn.relu),
    Flatten(),
    Dense(512),
    Dense(3, activation=tf.nn.softmax)
])

In [12]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])